In [1]:
import pandas as ps
import numpy as np 
import os
import re

In [3]:
folder = []
for item in os.listdir('data'):
    if(len(item.split('.')) == 1):
        folder.append(item)

In [4]:
folder

['Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 17 1974 Amaranthaceae',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae']

# =============================

## Extract species names

In [13]:
def especes_pages(path):
    pages = []
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.readlines()
            #print("=================")
            for j in range(len(lines)):
                l = lines[j].strip()
                l = l.replace('  ',' ')
                l = f_remove_accents(l)
                x = re.search("cle.?\sdes\sespece", l)
                if(x):
                    pages.append(i+1)
                    pages.append(i+2)
                    pages.append(i+3)
    return pages

In [15]:
def get_especes(pages):
    especes = []
    for p in pages:
        with open(path+'/'+ str(p) +'.txt', 'r') as fp:
            lines = fp.readlines()
            for j in range(len(lines)):
                l = lines[j].strip()
                l = l.replace('  ',' ')
                l = f_remove_accents(l)
                x = re.findall("\s\w\.\s\w+$", l)
                x = re.findall("[A-Za-z0-9]\.\s\w\.\s\w+\.?$", l)
                if(x):
                    especes.append(x)
    especes = np.array(especes).reshape(-1)
    esp = [esp.strip().split(' ')[2][:-1] for esp in especes]
    initials = [esp.strip().split(' ')[1][:-1].upper() for esp in especes]
    return esp, initials

def preprocess(line):
    l = line.strip()
    l = l.replace('  ',' ')
    #l = f_remove_accents(l)
    return l

## get species full name

In [21]:
def get_especes_names(especes, initials, path):
    esp_names = []
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.readlines()
            for j in range(len(lines)):
                l = preprocess(lines[j])
                for i, esp in enumerate(especes):
                    contains = l.find(esp.capitalize()) != -1 or l.find(esp) != -1 
                    if contains:
                        exp = "^[1-9]\.\s{}[a-z]+.*".format(initials[i])
                        x = re.findall(exp, l)
                        if(x):
                            full_name = x[0].strip()[3:]
                            full_name = full_name.split(' ')[0]+' '+full_name.split(' ')[1]
                            esp_names.append([full_name,x[0]])
    return esp_names

## Execute

In [28]:
paths = ['data/'+ fold for fold in folder]
names = []
for path in paths:
    print(path)
    print("======================")
    pages = especes_pages(path)
    especes, initials = get_especes(pages)
    print("Species number : "+ str(len(especes)))
    all_species = get_especes_names(especes,initials, path)
    names += all_species
    print("Species title number : "+ str(len(all_species)))

data/Flore du Cameroun tome 10 1970 Ombellales
Species number : 15
Species title number : 14
data/Flore du Cameroun tome 12 1972 Loganiaceae
Species number : 63
Species title number : 43
data/Flore du Cameroun tome 16 1973 Sapindaceae
Species number : 61
Species title number : 41
data/Flore du Cameroun tome 17 1974 Amaranthaceae
Species number : 21
Species title number : 17
data/Flore du Cameroun tome 2 1964 Sapotacees
Species number : 33
Species title number : 33
data/Flore du Cameroun tome 6 1967 Cucurbitaceae
Species number : 37
Species title number : 30
data/Flore du Cameroun tome 9 1970 Caesalpinioidae
Species number : 90
Species title number : 66


In [29]:
import pandas as pd
df=pd.DataFrame(names,columns=["espèce","description"])

In [31]:
df.to_csv('flora_titles_with_species_names.csv',index=False)

In [34]:
data=pd.read_excel('flora_titles_with_species_names.xlsx')
data

,espèce,description
0,Schefflera abyssinica,1. Schefflera abyssinica
1,Schefflera Mannii,3. Schefflera Mannii (Hooker fil.) Harms
2,Hydrocotyle sibthorpioides,1. Hydrocotyle sibthorpioides Lamarck
3,Hydrocotyle hirta,2. Hydrocotyle hirta R. Brown ex A. Richard
4,Hydrocotyle bonariensis,3. Hydrocotyle bonariensis Lamarck
...,...,...
239,Monopetalanthus Pellegrinii,5. Monopetalanthus Pellegrinii A. Chevalier
240,Swartzia fistuloides,1. Swartzia fistuloides Harms
241,Swartzia madagascariensis,2. Swartzia madagascariensis Desvaux
242,Csesalpinia Welwitschiana,1. Csesalpinia Welwitschiana (Oliver) Brenan


In [36]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

In [40]:
data.reset_index(drop=True,inplace=True)

In [41]:
data

,espèce,description
0,Schefflera abyssinica,1. Schefflera abyssinica
1,Schefflera Mannii,3. Schefflera Mannii (Hooker fil.) Harms
2,Hydrocotyle sibthorpioides,1. Hydrocotyle sibthorpioides Lamarck
3,Hydrocotyle hirta,2. Hydrocotyle hirta R. Brown ex A. Richard
4,Hydrocotyle bonariensis,3. Hydrocotyle bonariensis Lamarck
...,...,...
200,Monopetalanthus Pellegrinii,5. Monopetalanthus Pellegrinii A. Chevalier
201,Swartzia fistuloides,1. Swartzia fistuloides Harms
202,Swartzia madagascariensis,2. Swartzia madagascariensis Desvaux
203,Csesalpinia Welwitschiana,1. Csesalpinia Welwitschiana (Oliver) Brenan


In [42]:
data.to_csv('species_and_titles_dataframe.csv',index=False)